In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=be4cc0eb4cb22a87744b5d1bc93fcf6b3e3f5d30e592bca547df1d63ac3394e6
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

# Spark entry point
spark = SparkSession \
    .builder \
    .appName("Aula Interativa 2 - Apache Spark") \
    .getOrCreate()

spark.version


'3.3.1'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
titanic_df = spark.read.csv('/content/drive/MyDrive/MBA/titanic-3.csv',header='True',inferSchema='True')

titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
titanic_df.count()

891

In [ ]:
titanic_df.groupBy('survived').count().show()

+--------+-----+
|survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [11]:
titanic_df.createOrReplaceTempView('table')

spark.sql("SELECT survived, count(*) FROM table GROUP BY survived").show()

+--------+--------+
|survived|count(1)|
+--------+--------+
|       1|     342|
|       0|     549|
+--------+--------+



In [ ]:
titanic_df.groupBy('survived').agg({"Fare": "avg"}).show()

+--------+------------------+
|survived|         avg(Fare)|
+--------+------------------+
|       1| 48.39540760233917|
|       0|22.117886885245877|
+--------+------------------+



In [ ]:
spark.sql("SELECT survived, avg(fare) FROM table GROUP BY survived").show()

+--------+------------------+
|survived|         avg(fare)|
+--------+------------------+
|       1| 48.39540760233917|
|       0|22.117886885245877|
+--------+------------------+



In [ ]:
spark.sql("SELECT age, count(*) as age_count FROM table WHERE survived == 1 GROUP BY age ORDER BY age_count DESC").show()


+----+---------+
| age|age_count|
+----+---------+
|null|       52|
|24.0|       15|
|35.0|       11|
|27.0|       11|
|36.0|       11|
|22.0|       11|
|30.0|       10|
|18.0|        9|
|32.0|        9|
|19.0|        9|
|31.0|        8|
|29.0|        8|
| 4.0|        7|
|28.0|        7|
|34.0|        6|
|25.0|        6|
|42.0|        6|
|40.0|        6|
|48.0|        6|
|33.0|        6|
+----+---------+
only showing top 20 rows



In [16]:
# UDFs ajudam plugar funções complexas no SQL.
from pyspark.sql.functions import udf

def uppercase(str):
    return str.upper()

def ml_model(data):
    return model(data)

def maior_idade(age):
  return age is None or age > 18

spark.udf.register("uppercase", uppercase)
spark.udf.register("maior_idade", maior_idade)

spark.sql("SELECT uppercase(Name), Name from table WHERE maior_idade(Age) == TRUE").show(5)

+--------------------+--------------------+
|     uppercase(Name)|                Name|
+--------------------+--------------------+
|BRAUND, MR. OWEN ...|Braund, Mr. Owen ...|
|CUMINGS, MRS. JOH...|Cumings, Mrs. Joh...|
|HEIKKINEN, MISS. ...|Heikkinen, Miss. ...|
|FUTRELLE, MRS. JA...|Futrelle, Mrs. Ja...|
|ALLEN, MR. WILLIA...|Allen, Mr. Willia...|
+--------------------+--------------------+
only showing top 5 rows



In [23]:
# https://sparkbyexamples.com/spark/spark-write-dataframe-to-csv-file/

spark.sql("SELECT uppercase(Name), Age from table").write.option("header", True).format("csv").save("/content/drive/MyDrive/MBA/names")

In [25]:
# https://sparkbyexamples.com/spark/spark-read-and-write-json-file/


spark.sql("SELECT age, count(*) as age_count FROM table WHERE survived == 1 GROUP BY age ORDER BY age_count DESC").write.format("json").save("/content/drive/MyDrive/MBA/ages")

In [ ]:
# https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html
    

# Saving data to a JDBC source
#jdbcDF.write \
#    .format("jdbc") \
#    .option("url", "jdbc:postgresql:dbserver") \
#    .option("dbtable", "schema.tablename") \
#    .option("user", "username") \
#    .option("password", "password") \
#    .save()
    